In [1]:
import numpy as np
import cv2
import pyautogui
import time
from matplotlib import pyplot as plt

In [2]:
#getting first frame
imgPrev = pyautogui.screenshot()
imgPrev = cv2.cvtColor(np.array(imgPrev), cv2.COLOR_RGB2BGR)

#defining number of screen to be divided into     
div = 10

x, y, z = imgPrev.shape
xit = x//div
yit = y//div

Histogram_comparison_threshold = 0.85
Danger_Frame_Limit = 4

protection = cv2.imread("Seizure_W.jpg")

DetectionRecord = []

In [3]:
def tic():
    global t
    t = time.time()

def toc():
    global t
    elapsed = time.time() - t
    #print(elapsed," Seconds Elapsed")

In [4]:
def ScanScreen(imgO, imgN):
    dangerMap = np.zeros((div,div),dtype = np.bool)
    
    #Iterating through the screen divisions
    for X in range(0, div):
        for Y in range(0, div):
            
            histS1 = cv2.calcHist([imgO[X*div:(X+1)*div,Y*div:(Y+1)*div]],[0],None,[256],[0,256])
            histS2 = cv2.calcHist([imgN[X*div:(X+1)*div,Y*div:(Y+1)*div]],[0],None,[256],[0,256])
            #print("Histograms img1=", histS1, " img2=", histS2)
        
            #sc = cv2.compareHist(histS1, histS2, cv2.HISTCMP_INTERSECT)
            sc = cv2.compareHist(histS1, histS2, cv2.HISTCMP_BHATTACHARYYA)
            #print("Compared Hist at X=", X, " Y=", Y, " : ", sc)
            dangerMap[X,Y] = sc > Histogram_comparison_threshold
            
    return dangerMap

In [5]:
def ScanRecord(record):
    DangerCounts = np.zeros((div, div),dtype = np.uint8)
    
    for dMap in record:
        DangerCounts = np.add(DangerCounts, dMap.astype(int))
    
    return np.any(np.greater(DangerCounts, Danger_Frame_Limit))

In [14]:
#getting first frame
imgPrev = pyautogui.screenshot()
imgPrev = cv2.cvtColor(np.array(imgPrev), cv2.COLOR_RGB2BGR)

while(True):  
    tic()
    #getting new frame
    imgNew = pyautogui.screenshot()
    imgNew = cv2.cvtColor(np.array(imgNew), cv2.COLOR_RGB2BGR)
    
    #checking if screenshot exists
    if imgNew is not None:
        #cv2.imshow("screenshots", imgNew)
        DetectionRecord.append(ScanScreen(imgPrev, imgNew))
        if len(DetectionRecord) > 10:
            DetectionRecord.pop(0)
            #checking if record has more danger frames than limit
            if ScanRecord(DetectionRecord):
                # PREVENT
                
                print("prevented")
                cv2.imshow("Seizure Prevention", protection)
                
                #DetectionRecord.clear()
                DetectionRecord = []
            
        
        imgPrev = imgNew
        if cv2.waitKey(1) & 0xFF == ord('q'):
#             cv2.destroyAllWindows()
#             continue
            break
    
    else:
        print("No new frame captured")
        
    toc()

#cv2.waitKey(0)
cv2.destroyAllWindows()

prevented
prevented
